In [1]:
import json
import os
import random
import copy

In [24]:
# file_path = "data/instruction/"
train_files = os.listdir()
train_files = [i for i in train_files if "G" in i and "_" in i]
train_files

['G1_instruction.json',
 'G3_instruction.json',
 'G2_instruction.json',
 'G2_category.json',
 'G1_tool.json',
 'G1_category.json']

In [80]:
all_data = []

# for i in train_files:
test_set = "G1_instruction"
with open("{}.json".format(test_set), 'r') as file:
        # Load the JSON data
        data = json.load(file)
all_data.append(data)

In [5]:
human_interaction_tool = {
                            "category_name": "QAQ",
                            "tool_name": "qaq",
                            "api_name": "get_qaq",
                            "api_description": "Conditionally requests user for an input, and returns their input or a predefined message stating no user input is required for answering the query.",
                            "required_parameters": [
                                                        {
                                                            "name": "requires_input",
                                                            "type": "int",
                                                            "description": "A flag to determine if user input is needed (1) or not (0)",
                                                            "default": 0
                                                        }
                                                    ],
                            "optional_parameters": [
                                                        {
                                                            "name": "input_question",
                                                            "type": "string",
                                                            "description": "The question to ask the user to gather more information for answering the query",
                                                            "default": ""
                                                        }
                                                    ],
                            "method": "GET",
                            "template_response": {"message": "str"}
                        }

relevant_human_tool = [
                "qaq",
                "get_qaq"
            ]

# Missing info queries

In [ ]:
example_missing_info_queries = ["Recommend movies similar to those I like",
                                "Translate my name to English",
                                "Describe the song from its lyrics",
                                "Find data for the country code"]

# Masking completable queries

tool list
- Advertising/ai_quote_laksh
- Artificial_Intelligence_Machine_Learning/ai_airport_code_extractor
- Artificial_Intelligence_Machine_Learning/ebook_ai
- Business/email_linkedin
- Business/email_linkedin

# Masking Incompletable queries

Tool list
- Advertising/bing_keyword_planner
- Advertising/musk_plane
- Artificial_Intelligence_Machine_Learning/stable_diffusion
- Artificial_Intelligence_Machine_Learning/ai_url_shortener
- Artificial_Intelligence_Machine_Learning/youtube_summarizer_by_chatgpt
- Business/html_to_pdf
- Business/search_properties
- Business_software/password_checker

In [88]:
def get_relevant_queries(tools,data):

    all_apis = set()
    # tool_match = [(i.split("/")[0].lower().replace("_", " "), i.split("/")[1].lower().replace("_", " ")) for i in tools]
    relevant = []
    non_relevant = []
    for i in data:
        apis = i["api_list"]
        found = False
        for api in apis:
            all_apis.add((api["category_name"].lower(), api["tool_name"].lower()))
            if (api["category_name"].lower(), api["tool_name"].lower()) in tools:        
                relevant.append(i)
                found = True
                break
        if not found:
            non_relevant.append(i)


    return relevant, non_relevant

def clean_queries(queries, tools, add=False):
    q_copy = copy.copy(queries)

    names = [t[1] for t in tools]
    for count, i in enumerate(queries):
        apis = i["api_list"]
        relevant = i["relevant APIs"]
        final_apis = []
        final_relevant = []
        
        for api in apis:
            if (api["category_name"].lower(), api["tool_name"].lower()) not in tools:
                final_apis.append(api)

        for rel_api in relevant:
            name = rel_api[0]
            if name not in names:
                final_relevant.append(rel_api)
            
        
        if add:
            final_apis.append(human_interaction_tool)
            final_relevant.append(relevant_human_tool)

        q_copy[count]["api_list"] = final_apis
        q_copy[count]["relevant APIs"] = final_relevant

    return q_copy


def save_random_subest(queries, name):
    # for i in range(len(queries)):
        # queries[i]["query_id"] = i+1
    with open(name, 'w') as file:
        json.dump(queries, file,indent=4)

In [66]:
all_apis = set()

for d in all_data:
    for i in d:
        apis = i["api_list"]
        
        for api in apis:
            all_apis.add((api["category_name"].lower().replace("_", " "), api["tool_name"].lower().replace("_", " ")))


# f = open("all_apis.txt", "w")

# for i in all_apis:
#     f.write(str(i) + "\n")
# f.close()

# random_subset = random.sample(all_apis, 10)

tools = [('tools', 'email validation'),
('tools', 'domain name search'),
('tools', 'story'),
('tools', 'famous  quotes'),
('sms', 'whatsapp checker pro'),
('email', 'mailcheap'),
('media', 'bing image search'),
('media', 'placekitten'),
('media', 'open library'),
('media', 'colorful'),
('media', 'convm'),
('advertising', 'lets'),
('advertising', 'test v6'),
('advertising', 'underscore test'),
('ecommerce', 'amazon data v2'),
('ecommerce', 'natural milk'),
('ecommerce', 'pricer'),
('finance', 'date calculator'),
('finance', 'exchange rates live'),
('finance', 'inflation by api-ninjas'),
('finance', 'oil thai price'),
('finance', 'sales tax by api-ninjas'),
('sports', 'football (soccer) team names'),
('gaming', 'epic free games'),
('gaming', 'dice roll simulator'),
('travel', 'cities cost of living'),
('travel', 'biggest cities'),
('travel', 'gas price'),
('travel', 'ranked crime cities'),
('weather', 'air quality api')
]

random_subset = set()
for i in tools:
    random_subset.add(i)



random_subset_2 = random.sample(all_apis.difference(random_subset), 60)


/tmp/ipykernel_1797018/3509065346.py:57: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_subset_2 = random.sample(all_apis.difference(random_subset), 60)


In [81]:
relevant_queries = []
non_relevant_queries = []

for i in all_data:
    # random_subset.union(random_subset_2)
    r1, r2 = get_relevant_queries(random_subset.union(random_subset_2),i)
    relevant_queries.extend(r1)
    non_relevant_queries.extend(r2)


solvable = []
unsolvable = []
# for i in relevant_queries:
r1, r2 = get_relevant_queries(random_subset, relevant_queries)
solvable.extend(r1)
unsolvable.extend(r2)


In [82]:
# print(len(relevant_queries), len(non_relevant_queries))
print(len(solvable), len(unsolvable), len(non_relevant_queries))

2 58 140


In [76]:
sample_noqaq = clean_queries(random.sample(non_relevant_queries,36), set(), add=False)
sample_qaq = clean_queries(random.sample(non_relevant_queries,36), set(), add=True)

save_random_subest(sample_noqaq, "{}_type4_no_qaq.json".format(test_set))
save_random_subest(sample_qaq, "{}_type4_qaq.json".format(test_set))

not adding
[{'category_name': 'Translation', 'tool_name': 'Translator', 'api_name': 'Get Languages', 'api_description': 'Get all supported languages', 'required_parameters': [], 'optional_parameters': [], 'method': 'GET', 'template_response': {'messages': 'str', 'info': 'str'}}, {'category_name': 'QAQ', 'tool_name': 'qaq', 'api_name': 'get_qaq', 'api_description': 'Conditionally requests user for an input, and returns their input or a predefined message stating no user input is required for answering the query.', 'required_parameters': [{'name': 'requires_input', 'type': 'int', 'description': 'A flag to determine if user input is needed (1) or not (0)', 'default': 0}], 'optional_parameters': [{'name': 'input_question', 'type': 'string', 'description': 'The question to ask the user to gather more information for answering the query', 'default': ''}], 'method': 'GET', 'template_response': {'message': 'str'}}]
not adding
[{'category_name': 'Mapping', 'tool_name': 'SpatialScale Route Engin

In [55]:
sample_noqaq = clean_queries(unsolvable, random_subset_2, add=False)
sample_qaq = clean_queries(unsolvable, random_subset_2, add=True)


save_random_subest(random.sample(sample_noqaq,36), "{}_type2_no_qaq.json".format(test_set))
save_random_subest(random.sample(sample_qaq,36), "{}_type2_qaq.json".format(test_set))

In [83]:
sample_noqaq = clean_queries(solvable, random_subset, add=False)
# sample_qaq = clean_queries(solvable, random_subset, add=True)


save_random_subest(random.sample(sample_noqaq, len(sample_noqaq)), "{}_type1_no_qaq.json".format(test_set))
# save_random_subest(random.sample(sample_qaq, len(sample_qaq)), "{}_type1_qaq.json".format(test_set))

not adding
[]
not adding
[]


In [ ]:
# root = "data/toolenv/tools/"
# categories = os.listdir(root) 
# cat_lower = [i.lower().replace("_", " ") for i in os.listdir(root)]

# allpaths = []
# for api in all_apis:
#     ind = cat_lower.index(api[0])
   
#     cat = categories[ind]
#     path = root + cat + "/"
#     print(path)

#     tools = os.listdir(path)
#     tools_lower = [i.lower().replace("_", " ") for i in tools]
    
    
#     f = api[1].replace("-", " ")
#     f = f.replace("&", "")
#     apiInd = tools_lower.index(f)
    
#     finPath = path + tools[apiInd]
#     allpaths.append(finPath)
    

In [109]:
def save(test_set):
        all_data = []

        # for i in train_files:
        
        with open("{}.json".format(test_set), 'r') as file:
                # Load the JSON data
                data = json.load(file)
        all_data.append(data)


        all_apis = set()

        for d in all_data:
                for i in d:
                        apis = i["api_list"]
                        
                        for api in apis:
                                all_apis.add((api["category_name"].lower().replace("_", " "), api["tool_name"].lower().replace("_", " ")))


        tools = [('tools', 'email validation'),
        ('tools', 'domain name search'),
        ('tools', 'story'),
        ('tools', 'famous  quotes'),
        ('sms', 'whatsapp checker pro'),
        ('email', 'mailcheap'),
        ('media', 'bing image search'),
        ('media', 'placekitten'),
        ('media', 'open library'),
        ('media', 'colorful'),
        ('media', 'convm'),
        ('advertising', 'lets'),
        ('advertising', 'test v6'),
        ('advertising', 'underscore test'),
        ('ecommerce', 'amazon data v2'),
        ('ecommerce', 'natural milk'),
        ('ecommerce', 'pricer'),
        ('finance', 'date calculator'),
        ('finance', 'exchange rates live'),
        ('finance', 'inflation by api-ninjas'),
        ('finance', 'oil thai price'),
        ('finance', 'sales tax by api-ninjas'),
        ('sports', 'football (soccer) team names'),
        ('gaming', 'epic free games'),
        ('gaming', 'dice roll simulator'),
        ('travel', 'cities cost of living'),
        ('travel', 'biggest cities'),
        ('travel', 'gas price'),
        ('travel', 'ranked crime cities'),
        ('weather', 'air quality api')
        ]

        random_subset = set()
        for i in tools:
                random_subset.add(i)


        diff = all_apis.difference(random_subset)
        random_subset_2 = random.sample(diff, int(0.3*len(diff)))

        relevant_queries = []
        non_relevant_queries = []

        for i in all_data:
        # random_subset.union(random_subset_2)
                r1, r2 = get_relevant_queries(random_subset.union(random_subset_2),i)
                relevant_queries.extend(r1)
                non_relevant_queries.extend(r2)


        solvable = []
        unsolvable = []
        # for i in relevant_queries:
        r1, r2 = get_relevant_queries(random_subset, relevant_queries)
        solvable.extend(r1)
        unsolvable.extend(r2)
        print(len(solvable), len(unsolvable), len(non_relevant_queries))

        return (solvable, unsolvable, non_relevant_queries)


       



In [117]:
test_set = "G2_instruction"


solvable, unsolvable, non_relevant_queries = save(test_set)
sample_noqaq = clean_queries(non_relevant_queries, set(), add=False)
save_random_subest(random.sample(sample_noqaq, min(36, len(sample_noqaq))), "{}_type4_no_qaq.json".format(test_set))

solvable, unsolvable, non_relevant_queries = save(test_set)
sample_qaq = clean_queries(non_relevant_queries, set(), add=True)
save_random_subest(random.sample(sample_qaq,min(36, len(sample_qaq))), "{}_type4_qaq.json".format(test_set))



solvable, unsolvable, non_relevant_queries = save(test_set)
sample_noqaq = clean_queries(unsolvable, random_subset_2, add=False)
save_random_subest(random.sample(sample_noqaq,min(36, len(sample_noqaq))), "{}_type2_no_qaq.json".format(test_set))

solvable, unsolvable, non_relevant_queries = save(test_set)
sample_qaq = clean_queries(unsolvable, random_subset_2, add=True)
save_random_subest(random.sample(sample_qaq,min(36, len(sample_qaq))), "{}_type2_qaq.json".format(test_set))


solvable, unsolvable, non_relevant_queries = save(test_set)
sample_noqaq = clean_queries(solvable, random_subset, add=False)
save_random_subest(random.sample(sample_noqaq, min(36, len(sample_noqaq))), "{}_type1_no_qaq.json".format(test_set))

solvable, unsolvable, non_relevant_queries = save(test_set)
sample_qaq = clean_queries(solvable, random_subset, add=True)
save_random_subest(random.sample(sample_qaq, min(36, len(sample_qaq))), "{}_type1_qaq.json".format(test_set))

24 100 76
24 108 68
24 93 83
24 108 68
24 94 82
24 101 75


/tmp/ipykernel_1797018/2793521881.py:60: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_subset_2 = random.sample(diff, int(0.3*len(diff)))
